# Red Wine Quality Prediction 

authors: 

## Summary 

## Introduction 

## EDA

### Imports

In [3]:
import pandas as pd 
import numpy as np
import sys
from hashlib import sha1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler

### Part 1: Dataset Description
The dataset which will be used for analysis is the "winequality-red.csv" file from the <a href=https://archive.ics.uci.edu/dataset/186/wine+quality>UC Irvine Machine Learning Repository</a>, which was originally referenced from <a href=http://www3.dsi.uminho.pt/pcortez/wine/>Decision Support Systems, Elsevier</a>. The dataset contains physiochemical proprties (features) of red vinho verde wine samples from the north of Portugal, along with an associated wine quality score from 1 (worst) to 10 (best). 

### Part 2: Data Summary
In this section the df.info() and df.describe() commands will be used to obtain a preliminary overview of the dataset. Areas of particular interest are row and column counts, target variable type, feature types, and null entries. 

In [4]:
# Import the dataset:
df = pd.read_csv('data/winequality-red.csv', sep = ';')

# Visualize the first 5 rows:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
# Determine the size of the dataset:
print(df.shape)

# Determine the value counts for each unique target (quality) value::
print(df.quality.unique())
df.quality.value_counts().sort_index()

(1599, 12)
[5 6 7 4 8 3]


quality
3     10
4     53
5    681
6    638
7    199
8     18
Name: count, dtype: int64

It can be seen that there are 11 feature columns representing physiochemical characteristics of the wines, such as fixed acidity, residual sugar, chlorides, density, etc. There are 1598 rows or observations in the dataset. The target is the quality column which is listed as a set of ordinal values from 3 to 8. Most observations have an "average" quality between 5 and 7, with very few observations below a score of 5 or above a score of 7. This shows an unbalanced dataset.

In [6]:
# View feature dtypes and counts, null values:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In each of the 11 feature columns above, all entries have non-null values. As a result, imputation is not required to replace the null values. The data types of all features columns are float64 and the target column is int64. Although the target column "quality" is numeric, this problem could be considered as a classification or regression problem, as the numeric targets represent a wine quality category (i.e. from 1 to 10). Lastly, due to the numeric nature of the features preprocessing in the form of StandardScaler() will be required to scale each of the features, to prevent features of a large relative magnitude from dominating features of a smaller relative magnitude. 

In [7]:
# View summary statistics:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


The summary statistics show that discrepancy in magnitude between each of the classes, which will be addressed with scaling during pre-processing. It can also be seen that several classes have outliers including fixed acidity (mean of 8.31, max of 15.90), residual sugar (mean of 2.53, max of 15.50), free sulfur dioxide (mean of 15.87, max of 72.00), and total sulfur dioxide (mean of 46.47, max of 289.00). The outliers will be explored further in subseqent analysis, as they may impact model prediction accuracy. 

## Methods

EDA VIZ

In [1]:
import pandas as pd 
import numpy as np
import altair as alt
import matplotlib.pyplot as plt


In [2]:
# read data
wine_df = pd.read_csv('data/winequality-red.csv', sep = ';')

wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


We first observe the distribution of the features using their statistical summaries and a histogram. We can see that the majority of features have a skewed distribution, with many containing outliers. Volatile acidity, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, and sulphates all have very extreme outliers. Decision trees, linear models, K-NN and SVM are all sensitive to outliers. Omitting outliers may be useful for the performance of these models. 

In [3]:
wine_df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [4]:
# Features 
feature_df = wine_df.drop('quality', axis=1)

feature_names = list(feature_df.columns)


alt.Chart(feature_df).mark_bar().encode(
     alt.X(alt.repeat()).type('quantitative').bin(maxbins=40),
     y='count()',
).properties(
    width=200,
    height=200
).repeat(
    feature_names, 
    columns=4
)

alt.RepeatChart(...)

In [22]:
# Box plots 
alt.data_transformers.enable('vegafusion')

alt.Chart(feature_df).mark_boxplot().encode(
     alt.X(alt.repeat()).type('quantitative')
).properties(
    width=200,
    height=200
).repeat(
    feature_names, 
    columns=4
)


alt.RepeatChart(...)

Next we look at potential correlations between the features using a correlation matrix. The correlation matrix ranges from dark green to indicate a strong positive correlation, white to indicate no correlation and dark pink to indicate a strong negative correlation. The features with the strongest correlations are summarized in the table below.

In [5]:

feature_df.corr(numeric_only=True).style.background_gradient('PiYG')


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
fixed acidity,1.000000,-0.256131,0.671703,0.114777,0.093705,-0.153794,-0.113181,0.668047,-0.682978,0.183006,-0.061668
volatile acidity,-0.256131,1.000000,-0.552496,0.001918,0.061298,-0.010504,0.076470,0.022026,0.234937,-0.260987,-0.202288
citric acid,0.671703,-0.552496,1.000000,0.143577,0.203823,-0.060978,0.035533,0.364947,-0.541904,0.312770,0.109903
residual sugar,0.114777,0.001918,0.143577,1.000000,0.055610,0.187049,0.203028,0.355283,-0.085652,0.005527,0.042075
chlorides,0.093705,0.061298,0.203823,0.055610,1.000000,0.005562,0.047400,0.200632,-0.265026,0.371260,-0.221141
free sulfur dioxide,-0.153794,-0.010504,-0.060978,0.187049,0.005562,1.000000,0.667666,-0.021946,0.070377,0.051658,-0.069408
total sulfur dioxide,-0.113181,0.076470,0.035533,0.203028,0.047400,0.667666,1.000000,0.071269,-0.066495,0.042947,-0.205654
density,0.668047,0.022026,0.364947,0.355283,0.200632,-0.021946,0.071269,1.000000,-0.341699,0.148506,-0.496180
pH,-0.682978,0.234937,-0.541904,-0.085652,-0.265026,0.070377,-0.066495,-0.341699,1.000000,-0.196648,0.205633
sulphates,0.183006,-0.260987,0.312770,0.005527,0.371260,0.051658,0.042947,0.148506,-0.196648,1.000000,0.093595


In [6]:
# Features with stronger correlations (|correlation coeff| > 0.6)
corr_df = feature_df.corr().unstack().reset_index()
corr_df.columns = ['feature_1', 'feature_2', 'correlation']
corr_top = corr_df[(abs(corr_df['correlation']) > 0.6)]
corr_top = corr_top[corr_top['feature_1'] != corr_top['feature_2']]

# get unique pairings
corr_top[['feature_1', 'feature_2']] = corr_top[['feature_1', 'feature_2']].apply(sorted, axis=1, result_type='expand')
unique_corr_df = corr_top[~corr_top.duplicated(subset=['feature_1', 'feature_2'])]

plots= []
for i in range(unique_corr_df.shape[0]):
    plot = alt.Chart(wine_df).mark_point(opacity=0.3).encode(
            alt.X(unique_corr_df.iloc[i,0]),
            alt.Y(unique_corr_df.iloc[i,1])
            ).properties(
                height = 200, 
                width = 200, 
            )
    plots.append(plot)

# adjust scale and plot 
grid_plot = alt.vconcat(
    alt.hconcat(plots[0], plots[1].encode(alt.X(unique_corr_df.iloc[1,0], scale=alt.Scale(domain=(0.95, 1.05))), 
                                          alt.Y(unique_corr_df.iloc[1,1]))),
    alt.hconcat(plots[2], plots[3])
)

# display results
display(unique_corr_df)

grid_plot.properties(
   title = 'Features with the Strongest Correlations'
)

,feature_1,feature_2,correlation
2,citric acid,fixed acidity,0.671703
7,density,fixed acidity,0.668047
8,fixed acidity,pH,-0.682978
61,free sulfur dioxide,total sulfur dioxide,0.667666


alt.VConcatChart(...)

The stronger correlated features are: fixed acidity and citric acid; fixed acidity and density; fixed acidity and pH; and lastly, total sulfur dioxide and free sulfur dioxide. All have a positive correlation, except for pH and fixed acidity. To reduce model dimensionality and redundancy we may wish to remove a feature that is highly correlated with another. (?)

## Results 

In [ ]:
import pandas as pd 
import numpy as np
import sys
from hashlib import sha1

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
df = pd.read_csv('data/winequality-red.csv', sep = ';')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
has_na = df.isna().any().any()
has_na

In [ ]:
X = df.drop(columns = ['quality'])
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 522)


In [ ]:
models1 = {
    "dummy": DummyClassifier(random_state=522), 
    "KNN": KNeighborsClassifier(),
    "RBF SVM": SVC(random_state=123), 
    'Ridge model':Ridge(),
    'linear SVC':SVC(kernel = 'linear'),
    'decision tree': DecisionTreeClassifier(),
}

In [ ]:
from sklearn.compose import make_column_transformer


results1 = []

for name, model in models1.items():
    pipeline =  make_pipeline(StandardScaler(), model) 
    scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, n_jobs=-1)
 #   mean_std_cross_val_scores
    results1.append({
        'model': name,
        'test_score': np.mean(scores['test_score']),
        'train_score': np.mean(scores['train_score']),
        'fit_time': np.mean(scores['fit_time']),
        'score_time': np.mean(scores['score_time'])
    })

results_df1 = pd.DataFrame(results1)
results_df1.set_index('model', inplace=True)
results_df1

In [ ]:
pipe_ridge = make_pipeline(StandardScaler(), Ridge())
pipe_ridge.fit(X_train, y_train)

coeffs = pipe_ridge.named_steps["ridge"].coef_

# Use df.columns to get the feature names if X_train was derived from df
coeff_df = pd.DataFrame(data=coeffs, index=X_train.columns, columns=["Coefficients"])
sorted_coeff_df = coeff_df.sort_values(by="Coefficients", ascending=False)

sorted_coeff_df

In [ ]:
#drop variables with small coefficients(< 0.05) (free sulfur dioxide, residual sugar, density, citric acid)
X = df.drop(columns = ['quality','free sulfur dioxide', 'residual sugar', 'density', 'citric acid'])
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 522)
results1 = []

for name, model in models1.items():
    pipeline =  make_pipeline(StandardScaler(), model) 
    scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, n_jobs=-1)
 #   mean_std_cross_val_scores
    results1.append({
        'model': name,
        'test_score': np.mean(scores['test_score']),
        'train_score': np.mean(scores['train_score']),
        'fit_time': np.mean(scores['fit_time']),
        'score_time': np.mean(scores['score_time'])
    })

results_df1 = pd.DataFrame(results1)
results_df1.set_index('model', inplace=True)
results_df1

In [ ]:
#ridge coefficient
pipe_ridge = make_pipeline(StandardScaler(), Ridge())
pipe_ridge.fit(X_train, y_train)

coeffs = pipe_ridge.named_steps["ridge"].coef_

# Use df.columns to get the feature names if X_train was derived from df
coeff_df = pd.DataFrame(data=coeffs, index=X_train.columns, columns=["Coefficients"])
sorted_coeff_df = coeff_df.sort_values(by="Coefficients", ascending=False)

sorted_coeff_df

# ANALYSIS:

In this study, we employed various machine learning models to predict the quality of wine based on its chemical properties. The models used included a Dummy model, K-Nearest Neighbors (KNN), Support Vector Machine (SVM) with Radial Basis Function (RBF) kernel, Ridge Regression, and Linear Support Vector Classification (SVC). These models were rigorously cross-validated with 5 folds to assess their performance. The Python programming language, along with essential packages such as Pandas,scikit-learn was instrumental in conducting this analysis.

The initial performance of each model, as measured by test scores, was as follows:

Dummy Model: 0.437059
KNN: 0.562917
RBF SVM: 0.615313
Ridge Model: 0.340925
Linear SVC: 0.566094

To improve model performance and streamline the feature set, we conducted a coefficient analysis. This analysis led to the exclusion of variables with coefficients less than 0.05, including variables within 'free sulfur dioxide', 'residual sugar', 'density', and 'citric acid'. The updated model performances were:

Dummy Model: 0.437059 (unchanged)
KNN: 0.571541
RBF SVM: 0.606740
Ridge Model: 0.344543
Linear SVC: 0.576244

The new coefficients for the remaining variables were:

Alcohol: 0.322230
Sulphates: 0.145794
Fixed Acidity: 0.011367
pH: -0.059857
Chlorides: -0.067214
Total Sulfur Dioxide: -0.089573
Volatile Acidity: -0.179928


This refined analysis suggests a more focused model, with the reduced feature set enhancing the predictive accuracy of certain models, notably the KNN and Linear SVC. The data utilized for this study encompass various physicochemical properties of wine, such as acidity, sulfur dioxide levels, and alcohol content, which are believed to influence its quality.
Based on the results, we will focus on decision tree model, RBF SVM, Ridge for hyperparameter optimizations

## Discussion 

## References 

1. https://archive.ics.uci.edu/dataset/186/wine+quality
2. http://www3.dsi.uminho.pt/pcortez/wine/